# Coding project: Connect4 agent

SASSON Charlotte - BERARD Paul - PUJOL Corentin

## Introduction

The assignment is to develop an agent for Connect4, as implemented in the PettingZoo library, using Reinforcement Learning methods.

https://pettingzoo.farama.org/environments/classic/connect_four/

Students need to form groups of 2 or 3 *or 4* students, and will deliver a zip containing the code and a short report on the methods used. Deadline: April 14th.

The report should contain a description of the methods used and a description of the results. Here are a few questions that should be adressed in some way in the report
What design choices did you make in the development of your algorithm?
Describe the training procedure, the structure of the code.
Describe how you chose important hyperparameters, what you understand from their impact.
How did you assess the quality of your agent?
Do you think the approach you implemented would fare well on more complex environments, like backgammon, chess, go, Starcraft?
How would you improve it if given more time or computational power?
Describe also the workflow and how you split the work in the group.


The code should be clear and legible, variables well-named, and commented when helpful for comprehension. The final code should implement a class Player, with a method get_action that takes a state as specified in the PettingZoo environment and returns an integer between 0 and 6.

Performance is not the main point: better to show understanding of the methods you used and their limits.  

Ideas to get started: 
MCTS : https://sites.ualberta.ca/~szepesva/papers/CACM-MCTS.pdf

Eligibility Traces:
https://www.bkgm.com/articles/tesauro/tdl.html
https://www.ai.rug.nl/~mwiering/GROUP/ARTICLES/learning-chess.pdf

## Dependancies handling

In [ ]:
# import pip 

# !pip install pettingzoo
# !pip install pygame
# !pip install numpy

In [ ]:
from pettingzoo.classic import connect_four_v3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output

# Using the PettingZoo environment

This notebook provides smalls chunks of code to get you started with the Connect4 project. You do not have to use this code in you final file, but you can if you wish to. 

In [ ]:
env = connect_four_v3.env(render_mode="rgb_array")

env.reset()

# The main difference with the standard gym api is the way the environment is queried. The `step` method return `None`. To get the data on the environment, use the `last` method
state, reward, terminated, truncated, info = env.last()

# state is a dictionary with two keys: observation and action_mask
print(
    state["observation"].shape
)  # Observation is a numpy array with three coordinates, indicating the positions of the pieces of of player 0 and 1 on the the board
print(state["observation"][:, :, 0])  # Where the pieces of player 0 are
print(state["observation"][:, :, 1])  # Where the pieces of player 1 are

print(state["action_mask"])  # an array showing whether the actions are legal or nots


In [ ]:
env.reset()
env.step(0)

state, reward, terminated, truncated, info = env.last()

print(
    state["observation"].shape
) 
print(state["observation"][:, :, 0])  
print(state["observation"][:, :, 1])  

print(state["action_mask"])

# Agents

Here are some implementations of trivial agents that you should be able to beat ultimately. 

In [ ]:
class RandomPlayer:
    def __init__(self, rng=None):
        if rng is None:
            self.rng = np.random.default_rng()
        else:
            self.rng = rng

        self.name = "Random Player"

    def get_action(self, obs_mask, epsilon=None):
        return self.random_choice_with_mask(np.arange(7), obs_mask["action_mask"])

    def random_choice_with_mask(self, arr, mask):
        masked_arr = np.ma.masked_array(arr, mask=1 - mask)
        if masked_arr.count() == 0:
            return None
        return self.rng.choice(masked_arr.compressed())


In [ ]:
class PlayLeftmostLegal:
    def __init__(self):
        self.name = "Left Player"

    def get_action(self, obs_mask, epsilon=None):
        for i, legal in enumerate(obs_mask["action_mask"]):
            if legal:
                return i
        return None


# Running a game


The following function runs a full game between the two agents. 

In [ ]:
def play_game(env, agent0, agent1, display=False):
    done = False
    env.reset()
    obs, _, _, _, _ = env.last()
    while not done:
        for i, agent in enumerate([agent0, agent1]):
            action = agent.get_action(obs, epsilon=0)
            env.step(action)
            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()
            obs, reward, terminated, _, _ = env.last()
            done = terminated
            if np.sum(obs["action_mask"]) == 0:
                if display: 
                    print('Draw')
                return 0.5
            if done:
                if display:
                    print(f"Player {i}: {agent.name} won")
                    print(obs['observation'][:, :, 0]- obs['observation'][:, :, 1])
                return i

In [ ]:
agent0 = RandomPlayer()
agent1 = PlayLeftmostLegal()

play_game(env, agent0, agent1, display=True)

In [ ]:
plt.hist([play_game(env, agent0, agent1, display=False) for _ in range(100)])
plt.show()

# Emulating a Gym environment

If we fix the opposite policy, the game from the point of view of the agent is equivalent to a Gym environment. The following class implements this simulation. Then any algorithm that would work in a gym environment with the same observations will work here. 

Note that we implemented the possibility to be the first or the second player. 

In [ ]:
class EnvAgainstPolicy: 
    def __init__(self, env, policy, first_player=True):
        self.policy = policy
        self.env = env
        self.first_player = first_player
        self.reset()

    def step(self, action):
        self.env.step(action)
        obs, reward, terminated, _, _ = self.env.last()
        if terminated: 
            self.last_step = obs, reward, True, False, {}
        else: 
            action = self.policy.get_action(obs)
            self.env.step(action)
            obs, reward, terminated, _, _ = self.env.last()
            self.last_step = obs, -reward, terminated, False, {}
        return self.last_step

    def reset(self):
        self.env.reset()
        if not(self.first_player): 
            obs, _, _, _, _ = self.env.last()
            action = self.policy.get_action(obs)
            self.env.step(action)

        self.last_step = self.env.last()
        return self.last_step

    def last(self):
        return self.last_step

# Evaluating an agent against a fixed policy: 

Using the environment above, we can evaluate the agent against this fixed policy. 

In [ ]:
def eval_against_policy(env, agent, policy, N_episodes=10, first_player=True):
    eval_env = EnvAgainstPolicy(env, policy, first_player=first_player)
    results = []
    for _ in range(N_episodes):
        done = False
        eval_env.reset()
        obs, _, _, _, _ = eval_env.last()
        while not done:
            action = agent.get_action(obs, epsilon=0)
            eval_env.step(action)
            obs, reward, done, _, _ = eval_env.last()
        results.append(reward)
    return results

We can see that if both players play randomly, there is a small but significant advantage to the first player. 

In [ ]:
plt.hist(eval_against_policy(env, RandomPlayer(), RandomPlayer(), N_episodes=1000, first_player=False))
plt.show()
plt.hist(eval_against_policy(env, RandomPlayer(), RandomPlayer(), N_episodes=1000, first_player=True))
plt.show()

# Your turn 

Try to build a decent agent. Be creative! You can try any idea that you have: the grade is not about performance of the agent, but more about illustrating phenomena happening in Reinforcement Learning for turn-based games. It's okay to 'help' the agent in any way, as long as it follows the ideas of RL (i.e., as long as there is some learning involved).




## Technical choices

Pour entraîner notre agent aux Connect4, nous avons choisi d'implémenter un algorithme de Q-Learning.

Fonctionnement de l'algorithme de Q-learning:

L'algorithme de Q-learning est une méthode de renforcement qui permet à un agent d'apprendre à prendre des décisions en fonction de la valeur attendue des récompenses futures.

Pour chaque état possible du jeu, l'agent maintient une table Q(s, a) des valeurs d'utilité attendues pour chaque action possible a, en supposant que l'agent suit une politique optimale.

L'agent choisit une action en fonction de l'état actuel du jeu, en sélectionnant l'action qui maximise la valeur attendue des récompenses futures.

L'agent met à jour la valeur de Q(s, a) en fonction de la récompense réelle qu'il reçoit pour cette action et de la valeur de Q(s+1, a+1) pour l'état  et l'action suivants, en utilisant la formule de mise à jour de Q-learning:

Q(s, a) <- Q(s, a) + alpha * (r + gamma * max(Q(s+1, a+1)) - Q(s, a))

Avec alpha le taux d'apprentissage, gamma le taux d'escompte (qui donne la priorité aux récompenses immédiates plutôt que futures), r est la récompense réelle pour l'action prise, et max(Q(s+1, a+1)) est la valeur maximale attendue des récompenses futures pour l'état suivant s+1 et toutes les actions possibles a+1.

## Initialisation des paramètres

In [ ]:
EPISODES = 10000
EPSILON = 0.1
ALPHA = 0.2
GAMMA = 0.9

## Initialisation de la table Q

La table Q contiendra les valeurs d'action-état pour chaque état et chaque action possible. Dans ce cas, il y a 2^42 états possibles (2 pour chaque cellule de la grille) et 7 actions possibles (une pour chaque colonne de la grille) :

In [ ]:
Q = np.zeros((2**21, 7), dtype='uint8')

## Entraînement de l'agent  dans l'environnement

Entraînement de l'agent en exécutant des épisodes dans l'environnement Connect Four.

In [ ]:
"""
Création d'une instance de l'environnement Connect Four V3 de PettingZoo. 

Cet environnement a deux agents, chacun jouant un pion différent.
Le but est d'aligner quatre pions de la même couleur sur une grille de 6x7 cases.
"""
env = connect_four_v3.env()

for i in range(EPISODES): #Un épisode correspond à une itération d'entraînement de l'agent (Il s'agit d'une partie complète)
    obs = env.reset() #On réinitialise l'environnement pour que l'agent à chaque itération apprenne à jouer d'un état initial de la grille
    done = False #Condition de sortie de la boucle (lorsque la partie est terminée)
    while not done:
        
        """
        Choix d'une action basée sur la politique epsilon-greedy
        
        Si le nombre aléatoire généré est inférieur à epsilon :
             - l'agent choisit une action aléatoire parmi les actions possibles

        Sinon :
            - il choisit l'action qui maximise la valeur Q de l'état actuel.
        """
        
        if np.random.uniform() < EPSILON: 
            action = np.random.choice(env.action_spaces["player_0"])
        else:
            state = np.ravel_multi_index((obs['board'], obs['mark']), (2, 3) * (6,))
            action = np.argmax(Q[state])
            
        #L'environnement effectue ensuite l'action choisie par l'agent et renvoie la nouvelle observation, la récompense, si l'épisode est terminé et des informations supplémentaires.
        new_obs, reward, done, _ = env.step(action)
        new_state = np.ravel_multi_index((new_obs['board'], new_obs['mark']), (2, 3) * (6,))
        
        # Update Q-table
        Q[state, action] += ALPHA * (reward + GAMMA * np.max(Q[new_state]) - Q[state, action])
        # Update state
        obs = new_obs